In [2]:
import pandas as pd
import sqlite3

In [3]:
excel_file_path = 'BD_SECTOR_INMOBILIARIO_01_2024.xlsx'
excel_data = pd.ExcelFile(excel_file_path)
sheet_names = excel_data.sheet_names
print(sheet_names)

['Índice', 'Venta', 'Renta']


In [4]:
venta_csv_path = 'venta_data.csv'
renta_csv_path = 'renta_data.csv'

venta_data = pd.read_excel(excel_file_path, sheet_name='Venta')
venta_data.to_csv(venta_csv_path, index=False)

renta_data = pd.read_excel(excel_file_path, sheet_name='Renta')
renta_data.to_csv(renta_csv_path, index=False)

print(f"Sheets converted to CSV:\n- Venta: {venta_csv_path}\n- Renta: {renta_csv_path}")

Sheets converted to CSV:
- Venta: venta_data.csv
- Renta: renta_data.csv


In [5]:
database_filename = 'real_estate.db'
cnn = sqlite3.connect(database_filename)

In [6]:
venta_df = pd.read_csv(venta_csv_path)
venta_df.to_sql('Venta', cnn, if_exists='replace', index=False)

renta_df = pd.read_csv(renta_csv_path)
renta_df.to_sql('Renta', cnn, if_exists='replace', index=False)

7011

In [7]:
%load_ext sql
%sql sqlite:///real_estate.db

Connecting to 'sqlite:///real_estate.db'

In [18]:
%%sql
SELECT 
    'Rental' AS Transaction_Type,
    r.Inmueble AS Property_Type,
    r.Municipio AS Municipality,
    r.Colonia AS Neighborhood,
    r.Precio AS Price,
    NULL AS Land_Area,
    NULL AS Construction_Area,
    NULL AS Price_per_SqMeter_Land,
    NULL AS Price_per_SqMeter_Construction
FROM 
    Renta r
UNION ALL
SELECT 
    'Sale' AS Transaction_Type,
    v.Inmueble AS Property_Type,
    v.Municipio AS Municipality,
    v.Colonia AS Neighborhood,
    v.Precio AS Price,
    v.[Metros cuadrados de terreno] AS Land_Area,
    v.[Metros cuadrados de construcción] AS Construction_Area,
    CASE 
        WHEN v.[Metros cuadrados de terreno] IS NOT NULL AND v.[Metros cuadrados de terreno] > 0 
        THEN v.Precio / v.[Metros cuadrados de terreno] 
        ELSE NULL 
    END AS Price_per_SqMeter_Land,
    CASE 
        WHEN v.[Metros cuadrados de construcción] IS NOT NULL AND v.[Metros cuadrados de construcción] > 0 
        THEN v.Precio / v.[Metros cuadrados de construcción] 
        ELSE NULL 
    END AS Price_per_SqMeter_Construction
FROM 
    Venta v;

Running query in 'sqlite:///real_estate.db'

Transaction_Type,Property_Type,Municipality,Neighborhood,Price,Land_Area,Construction_Area,Price_per_SqMeter_Land,Price_per_SqMeter_Construction
Rental,Departamento,TLAQUEPAQUE,SAN SEBASTIANITO,6800.0,None,None,None,None
Rental,Departamento,ZAPOPAN,PUERTA DE HIERRO,20000.0,None,None,None,None
Rental,Casa,ZAPOPAN,CIUDAD BUGAMBILIAS,25000.0,None,None,None,None
Rental,Casa,TLAQUEPAQUE,LA ASUNCION,20000.0,None,None,None,None
Rental,Casa,GUADALAJARA,AMERICANA,38000.0,None,None,None,None
Rental,Departamento,GUADALAJARA,AMERICANA,23000.0,None,None,None,None
Rental,Departamento,ZAPOPAN,LOMAS DEL VALLE,27000.0,None,None,None,None
Rental,Departamento,ZAPOPAN,None,18500.0,None,None,None,None
Rental,Casa,ZAPOPAN,CIUDAD DEL SOL,55000.0,None,None,None,None
Rental,Departamento,ZAPOPAN,PUERTA DEL VALLE,30000.0,None,None,None,None


In [22]:
df = pd.read_sql_query("""
SELECT 
    'Rental' AS Transaction_Type,
    r.Inmueble AS Property_Type,
    r.Municipio AS Municipality,
    r.Colonia AS Neighborhood,
    r.Precio AS Price,
    NULL AS Land_Area,
    NULL AS Construction_Area,
    NULL AS Price_per_SqMeter_Land,
    NULL AS Price_per_SqMeter_Construction
FROM 
    Renta r
UNION ALL
SELECT 
    'Sale' AS Transaction_Type,
    v.Inmueble AS Property_Type,
    v.Municipio AS Municipality,
    v.Colonia AS Neighborhood,
    v.Precio AS Price,
    v.[Metros cuadrados de terreno] AS Land_Area,
    v.[Metros cuadrados de construcción] AS Construction_Area,
    CASE 
        WHEN v.[Metros cuadrados de terreno] IS NOT NULL AND v.[Metros cuadrados de terreno] > 0 
        THEN v.Precio / v.[Metros cuadrados de terreno] 
        ELSE NULL 
    END AS Price_per_SqMeter_Land,
    CASE 
        WHEN v.[Metros cuadrados de construcción] IS NOT NULL AND v.[Metros cuadrados de construcción] > 0 
        THEN v.Precio / v.[Metros cuadrados de construcción] 
        ELSE NULL 
    END AS Price_per_SqMeter_Construction
FROM 
    Venta v;                  
""", cnn)

df.head(10)

,Transaction_Type,Property_Type,Municipality,Neighborhood,Price,Land_Area,Construction_Area,Price_per_SqMeter_Land,Price_per_SqMeter_Construction
0,Rental,Departamento,TLAQUEPAQUE,SAN SEBASTIANITO,6800.0,NaN,NaN,NaN,NaN
1,Rental,Departamento,ZAPOPAN,PUERTA DE HIERRO,20000.0,NaN,NaN,NaN,NaN
2,Rental,Casa,ZAPOPAN,CIUDAD BUGAMBILIAS,25000.0,NaN,NaN,NaN,NaN
3,Rental,Casa,TLAQUEPAQUE,LA ASUNCION,20000.0,NaN,NaN,NaN,NaN
4,Rental,Casa,GUADALAJARA,AMERICANA,38000.0,NaN,NaN,NaN,NaN
5,Rental,Departamento,GUADALAJARA,AMERICANA,23000.0,NaN,NaN,NaN,NaN
6,Rental,Departamento,ZAPOPAN,LOMAS DEL VALLE,27000.0,NaN,NaN,NaN,NaN
7,Rental,Departamento,ZAPOPAN,None,18500.0,NaN,NaN,NaN,NaN
8,Rental,Casa,ZAPOPAN,CIUDAD DEL SOL,55000.0,NaN,NaN,NaN,NaN
9,Rental,Departamento,ZAPOPAN,PUERTA DEL VALLE,30000.0,NaN,NaN,NaN,NaN


In [23]:
df.to_csv('real_estate_insights.csv', index=False)